# Bot twitter

Este bot genera reportes cada hora y se publican en twitter.

In [ ]:
"""
!pip install APIMakeSens
!pip install folium
!pip install tweepy
!pip install selenium
"""

### Cómo ejecutar este código cada hora:

- Descargar el notebook como un archivo .py
- Ejecutar en la terminal watch --interval=3600 python3 ruta/archivo.py

In [1]:
# Librerias
from MakeSens import MakeSens
import numpy as np
import datetime
import pytz
import folium
from folium.plugins import BeautifyIcon

In [2]:
# Rango de fecha
fechaFin = datetime.datetime.now(pytz.timezone('America/Bogota')).strftime('%Y-%m-%d %H:%M:%S') # Fecha y hora Colombia
#fechaInicio = fechaFin[:-8]+f'{int(fechaFin[-8:-6])-1:02}'+fechaFin[-6:] # Restar una hora

past_date =  datetime.datetime.now()- datetime.timedelta(hours=24)  #Restar 24 horas
fechaInicio = past_date.strftime('%Y-%m-%d %H:%M:%S')

print(fechaFin,fechaInicio)

2023-06-09 09:47:47 2023-06-08 09:47:47


In [3]:
# Datos de Estaciones
Estaciones = [('mE1_00004','Col. Tecnologico',7.1381572,-73.1180935),
              ('mE1_00005','Col. Santander',7.1385332,-73.121327),
              ('mE1_00006','Col. Pilar',7.108717813305484, -73.12173474310134),
              ('mE1_00007','Col. Normal',7.1277056,-73.116534),
              ('E2_00023','Grupo Halley',7.1393716,-73.1210968)]

In [4]:
Ntotal = int(24*30)

In [28]:
def get_pm25(Estaciones): # Obtener la media de pm25 de las estaciones
    pm25 = []
    for est in Estaciones:
        
        data = MakeSens.download_data(est[0],fechaInicio,fechaFin,'m')
        
        if not data.empty: # Revisa que la estación tenga datos en el rango horario indicado
            
            pm25_1 = np.array(data['pm25_1'])
            pm25_1[np.where(pm25_1>120)]=float("nan") # Igualar valores mayores a 100
            pm25_2 = np.array(data['pm25_2'])
            pm25_2[np.where(pm25_2>120)]=float("nan")
            Ndata = data['pm25_1'].count()
            
            if Ndata>0.5*Ntotal:     
                pm25_mean = np.mean([data['pm25_1'].mean(),data['pm25_2'].mean()]) # Media entre la media de ambas columnas de pm25
                pm25.append(pm25_mean)
            else:
                pm25.append(0.0)
        else:
            pm25.append(0.0)
    return pm25

In [31]:
pm25 = get_pm25(Estaciones)

mE1_00004 637 9.229199372056515 9.13657770800628
mE1_00005 698 12.146131805157593 13.064469914040115
mE1_00006 450 13.273333333333333 12.046666666666667
mE1_00007 428 15.228971962616823 14.544392523364486


Para guardar el mapa como imagen debe tenerse la plantilla en el mismo directorio que este código

In [32]:
# Coordenadas de Bucaramanga, Colombia

# Calcular centro entre estaciones extremas
latitudes = np.array(Estaciones)[:,2].astype('float64')
latitud = (np.max(latitudes)+np.min(latitudes))/2
longitudes = np.array(Estaciones)[:,3].astype('float64')
longitud = (np.max(longitudes)+np.min(longitudes))/2

#latitud = 7.1212675138485695
#longitud = -73.12578300721671

z = int(13-np.log2(10*(np.max(latitudes)-np.min(latitudes)))) # Zoom inicial en el mapa

# Crear el mapa de Bucaramanga con el estilo "CartoDB Positron"
bucaramanga_map = folium.Map(location=[latitud, longitud], 
                              zoom_start=z, tiles='CartoDB Positron',width=677,height=690, zoom_control=False)


def get_color(number):
    if number==0:
        return "#808080"  #black
    elif 0 < number <= 12:
        return '#00FF2E'  # Verde
    elif 13 <= number <= 35:
        return '#FFE000'  # Amarillo
    elif 38 <= number <= 55:
        return '#FF6100'  # Naranja
    elif 56 <= number:
        return '#FF0000'  # Rojo

    
for i in range(len(pm25)):
        
        if pm25[i] >= 0.0: # Valor crítico para alerta. Es 0 si se quieren mostrar todas las estaciones

            # Crear el ícono personalizado con un número
            icon_number = BeautifyIcon(
            border_color=get_color(pm25[i]),
            text_color='#000000',#get_color(pm25[i]),
            number=np.round(pm25[i],1),
            background_color='transparent', # Fondo transparente para que se vean estaciones cercanas entre sí
            icon_size=(40, 40),  # Tamaño del ícono en píxeles
            inner_icon_style='line-height:25px; font-size: 12px;',  # Ajusta el tamaño de la fuente aquí
            spin=True,
            border_width=2,
            )

            # Agregar marcador al mapa
            folium.Marker(
                [Estaciones[i][2], Estaciones[i][3]],
                popup=Estaciones[i][1],
                icon=icon_number
            ).add_to(bucaramanga_map)

bucaramanga_map

In [33]:
# Convertir mapa a imagen
bucaramanga_map.save('mapa.html')

import io
from PIL import Image

img_data = bucaramanga_map._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('mapa.png')

plantilla = Image.open('plantilla.jpeg') # Debe añadirse manualmente en la plantilla las estaciones a mostrar
plantilla.paste(img,(0,80)) # Se une la imagen generada con la plantilla

#new_image.save("images/merged_image.jpg","JPEG")
#print(full_image.size)
plantilla.show()

plantilla.save('mapa_full.png')

import os

os.remove('mapa.png')
os.remove('mapa.html')
os.remove('geckodriver.log')

In [9]:
import tweepy

# Llaves de acceso api de twitter

api_key = "VSBbc9ePnJEbqAIjYpy985m7T"
api_key_secret = "3DvgqIJSoGa97NlTv8JpCApKQEkdkfZuHxmdFplZJGJ1wovH5J"
access_token= '1651973312304627717-oSv22bNGgZ6VLKvFOX1pfTzju8jUEs'
access_token_secret= '7ENTTpeWdlXYFpqOq1efUV8DAdnHbkfLNDdivQ5t3yLvM'
bearer_token='AAAAAAAAAAAAAAAAAAAAAGiongEAAAAAi74ne7ihUqSAiBF4K%2Ft9JdeY8DE%3DPgLdwViyIFqUrKRpCkcVsPqkMG2btwJaDAdUF13vdiFwPIbhlK'

In [10]:
# Conexión api de twitter

client = tweepy.Client(bearer_token, api_key, api_key_secret, access_token, access_token_secret)
auth = tweepy.OAuth1UserHandler(api_key, api_key_secret, access_token, access_token_secret)
api = tweepy.API(auth)

In [11]:
tweet = f'''#CalidadAireBucaramanga Reporte de calidad de aire (material particulado 2.5 mm) {fechaFin} hora local.\n
Más información: https://class.redclara.net/halley/moncora/intro.html'''

media = api.media_upload('mapa_full.png')

client.create_tweet(text = tweet, media_ids=[media.media_id_string])

Response(data={'edit_history_tweet_ids': ['1667181820859895808'], 'id': '1667181820859895808', 'text': '#CalidadAireBucaramanga Reporte de calidad de aire (material particulado 2.5 mm) 2023-06-09 09:47:47 hora local.\n\nMás información: https://t.co/VDNNeGlk0O https://t.co/kPDWDHIifd'}, includes={}, errors=[], meta={})